In [1]:
!nvidia-smi

Thu Jan 23 10:05:04 2025       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 565.57.02              Driver Version: 566.03         CUDA Version: 12.7     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  NVIDIA GeForce RTX 4060 Ti     On  |   00000000:01:00.0  On |                  N/A |
|  0%   39C    P8              5W /  165W |     593MiB /  16380MiB |     10%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

In [1]:
import os
import torch
import transformers
import peft
import datasets
import evaluate
import time
assert torch.cuda.is_available(), "you need cuda for this part"
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')

/opt/conda/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
device

device(type='cuda', index=0)

In [2]:
dataset_path = "/app/datasets/oberon/qa/bb_ru/questions-qwen2.5-coder_7b-2025-21-1--09-23-51.json"
model_name = 'bb_ru_Cotype-Nano_20250123-100550'
model_path = f'/app/models/{model_name}'

EPOCHS = 7
bnb_config = transformers.BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_use_double_quant=True,
    bnb_4bit_compute_dtype=torch.bfloat16
)

In [3]:
tokenizer = transformers.AutoTokenizer.from_pretrained(model_path)


In [5]:
peft_config = peft.PeftConfig.from_pretrained(model_path)
model = transformers.AutoModelForCausalLM.from_pretrained(model_path, device_map=device,quantization_config=bnb_config,)
model = peft.PeftModel.from_pretrained(model, model_path, is_trainable=True)
model.print_trainable_parameters()

trainable params: 18,464,768 || all params: 1,562,179,072 || trainable%: 1.1820


In [20]:
qa_dataset = datasets.load_dataset("json", data_files=dataset_path)['train']
qa_dataset

Dataset({
    features: ['Source', 'Question', 'Answer'],
    num_rows: 5126
})

In [21]:
def format_chat(example):
    # Create a chat-like format using the tokenizer's chat template
    chat = [
        {"role": "user", "content": example["Question"]},
        {"role": "assistant", "content": example["Answer"]}
    ]
    # Apply the chat template
    formatted_chat = tokenizer.apply_chat_template(chat, tokenize=False)
    return {"formatted_chat": formatted_chat}

# Apply the formatting function to the dataset
qa_dataset = qa_dataset.map(format_chat)

# Print the first few examples to verify

print(qa_dataset[0]['formatted_chat'])
print('-' * 40)

Map: 100%|██████████| 5126/5126 [00:00<00:00, 13304.70 examples/s]

<|im_start|>user
Что такое Component Pascal?<|im_end|>
<|im_start|>assistant
Component Pascal - это язык программирования, который был разработан для создания многокомпонентных программ на системе BlackBox. Он является диалектом языка Pascal и расширяет его функциональность, добавляя поддержку компонентов.<|im_end|>

----------------------------------------


In [22]:
def tokenize_function(examples):
    res = tokenizer(examples["formatted_chat"], truncation=True, padding="max_length", max_length=512)
    res["labels"] = res["input_ids"].copy()
    return res

qa_dataset = qa_dataset.map(tokenize_function, batched=True)

Map: 100%|██████████| 5126/5126 [00:00<00:00, 5607.38 examples/s]


In [23]:
qa_dataset = qa_dataset.train_test_split(test_size=0.1).remove_columns(['Source', 'Question', 'Answer', 'formatted_chat'])
qa_dataset

DatasetDict({
    train: Dataset({
        features: ['input_ids', 'attention_mask', 'labels'],
        num_rows: 4613
    })
    test: Dataset({
        features: ['input_ids', 'attention_mask', 'labels'],
        num_rows: 513
    })
})

In [24]:
print(tokenizer.decode(qa_dataset["train"][99]["input_ids"]))

<|im_start|>user
Что делает команда `single[w] := TRUE;` внутри цикла?<|im_end|>
<|im_start|>assistant
Эта команда устанавливает значение элемента массива `single[w]` в `TRUE`. Массив `single` вероятно используется для отслеживания, какие люди уже были обработаны.<|im_end|>
<|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftex

In [25]:
timestr = time.strftime("%Y%m%d-%H%M%S")
data_collator = transformers.DataCollatorForLanguageModeling(tokenizer=tokenizer, mlm=False)
training_args = transformers.TrainingArguments(
    output_dir=f"./results_stage2/{model_name}/{timestr}/",
    eval_strategy="epoch",
    save_strategy="epoch",
    learning_rate=5e-5,
    per_device_train_batch_size=2,
    per_device_eval_batch_size=2,
    bf16=torch.cuda.get_device_capability(torch.cuda.current_device())[0] >= 8,  
    gradient_accumulation_steps=4,
    num_train_epochs=EPOCHS,
    weight_decay=0.01,
    #save_total_limit=2,
    logging_dir='./logs',
    logging_steps=32,
    warmup_steps=100, 
    #eval_steps=32, 
    #max_steps=512,
    
    #load_best_model_at_end=True,
)
trainer = transformers.Trainer(
    model=model,
    args=training_args,
    train_dataset=qa_dataset["train"],
    eval_dataset=qa_dataset["test"],
    data_collator=data_collator,
    #compute_metrics=compute_metrics,
)

In [ ]:
trainer.train()
#trainer.evaluate()

Epoch,Training Loss,Validation Loss


In [ ]:
trainer.state.log_history

In [28]:
for a in trainer.state.log_history:#. There is a parse_log_history
    print(f'{a['epoch']}:{a['loss']}')

0.055483311660164715:5.9775
0.11096662332032943:5.8955
0.16644993498049415:5.8022
0.22193324664065886:5.737
0.27741655830082357:5.9075
0.3328998699609883:5.7867
0.38838318162115304:5.6596
0.4438664932813177:5.7104
0.49934980494148246:5.899
0.5548331166016471:5.592
0.6103164282618119:5.3901
0.6657997399219766:5.3745
0.7212830515821413:5.1961
0.7767663632423061:5.3537
0.8322496749024707:5.2668
0.8877329865626354:5.4662
0.9432162982228002:5.2187
0.9986996098829649:5.3078


KeyError: 'loss'

In [125]:
import math
eval_results = trainer.evaluate()
print(f"Perplexity: {math.exp(eval_results['eval_loss']):.2f}")

Perplexity: 5.12


In [18]:
final_path = sft_model_path + f"_{timestr}"
model.save_pretrained(final_path)
tokenizer.save_pretrained(final_path)

2025-01-23 14:16:24,142 Resetting dropped connection: huggingface.co
2025-01-23 14:16:24,795 https://huggingface.co:443 "HEAD /MTSAIR/Cotype-Nano/resolve/main/config.json HTTP/11" 200 0
2025-01-23 14:16:24,982 https://huggingface.co:443 "HEAD /MTSAIR/Cotype-Nano/resolve/main/config.json HTTP/11" 200 0


('/app/models/bb_ru_Cotype-Nano_20250123-100550/tokenizer_config.json',
 '/app/models/bb_ru_Cotype-Nano_20250123-100550/special_tokens_map.json',
 '/app/models/bb_ru_Cotype-Nano_20250123-100550/vocab.json',
 '/app/models/bb_ru_Cotype-Nano_20250123-100550/merges.txt',
 '/app/models/bb_ru_Cotype-Nano_20250123-100550/added_tokens.json',
 '/app/models/bb_ru_Cotype-Nano_20250123-100550/tokenizer.json')

In [5]:
reference_model = transformers.AutoModelForCausalLM.from_pretrained(model_path, device_map=device,quantization_config=bnb_config,)

In [6]:
checkpoint = f"./results_stage2/{model_name}/20250123-153607/checkpoint-1152"
checkpoint_model = transformers.AutoModelForCausalLM.from_pretrained(checkpoint, device_map=device,quantization_config=bnb_config,)


In [9]:
from IPython.display import HTML, display
table_template = """<table style="border:1px solid black" >
  <tr>
    <th style="text-align: center; border:1px solid black">PROMPT</th>
    <th style="text-align: center; border:1px solid black">BEFORE</th>
    <th style="text-align: center; border:1px solid black">AFTER</th>
  </tr>
{}
</table>"""

row_template = '''  <tr>
    <td style="width:20%; border:1px solid black"><pre align="left">`{}`</pre></td>
    <td style="width:40%; border:1px solid black"><pre align="left">{}</pre></td>
    <td style="width:40%; border:1px solid black"><pre align="left">{}</pre></td>
  </tr>'''

def prompt_to_chat(prompt):
    messages = [
        {"role": "user", "content": prompt}
    ]
    text = tokenizer.apply_chat_template(
        messages,
        tokenize=False,
        add_generation_prompt=True
    )
    return text

def infer(model, prompt, l=100, use_chat = True, temperature=0.4, top_p = 0.8):
    if use_chat:
        prompt = prompt_to_chat(prompt)
    model_inputs = tokenizer([prompt], return_tensors="pt").to(model.device)

    generated_ids = model.generate(
        **model_inputs,
        max_new_tokens=l,
        temperature=temperature, 
        top_p=top_p,
        do_sample=True ,  
    )
    generated_ids = [
        output_ids[len(input_ids):] for input_ids, output_ids in zip(model_inputs.input_ids, generated_ids)
    ]
    
    response = tokenizer.batch_decode(generated_ids, skip_special_tokens=False)[0]
    return response


prompts =  [
    'Как в BlackBox можно вывести данные на консоль?', 
    'Какой тип данных в Component Pascal используется для хранения целых чисел?', 
    'МОДУЛЬ i21егэDemo2010C4ru;', 
    'Component Pascal is Oberon microsystems refinement of?', 
    'Log.String(', 
    'Типом целой константы является'
]  # feel free to add a few more that are not 100% assiciated with Python

In [10]:
def draw_diff(use_chat,l):
    print(f"USING CHAT TEMPLATE = {use_chat}")
        
    baseline = [infer(reference_model, p, l = l, use_chat=use_chat) for p in prompts]
    check = [infer(checkpoint_model, p, l = l, use_chat=use_chat) for p in prompts]
    rows = []
    for i, prompt in enumerate(prompts):
        # replace placeholders in the format() arguments
        rows.append(row_template.format(prompt, baseline[i], check[i]))
    display(HTML(table_template.format('\n'.join(rows))))

In [15]:
draw_diff(True, 100)

USING CHAT TEMPLATE = True


PROMPT,BEFORE,AFTER
`Как в BlackBox можно вывести данные на консоль?`,"Чтобы вывести данные на консоль, используйте процедуру StdLog.String. Это вызовет вывод строки в консоль, и если строка содержит символы, которые не могут быть отображены в консоли (например, символы Unicode), то они будут отображаться с помощью соответствующего кодового значения. Пример: MODULE i21примВывод; (* См. также: DevReferences *)","Для вывода данных на консоль используется функция `Log.String` для строк и `Log.Int` для целых чисел. Например, `Log.String(""Hello World"")` выводит строку ""Hello World"" на экран.<|im_end|>"
`Какой тип данных в Component Pascal используется для хранения целых чисел?`,"В языке программирования Component Pascal целые числа могут быть представлены как 32-битные (4 байта) или 64-битные (8 байт). В обоих случаях они представляют собой литерные цепочки, состоящие из одного символа. Например, число -10000000000000000000000000000000",Int16<|im_end|>
`МОДУЛЬ i21егэDemo2010C4ru;`,"Этот модуль предназначен для демонстрации работы с модулем i21eduЧерепашка. Для этого в нем реализованы следующие процедуры: PROCEDURE Печать ( VAR x: INTEGER ); Печатает число, переданное в качестве параметра. PROCEDURE Повторить; Повторяет печать чисел от 1 до 1000000 включительно. Повторение",Модуль содержит код для демонстрации работы с BlackBox.<|im_end|>
`Component Pascal is Oberon microsystems refinement of?`,"Oberon Microsystems was a company that developed the Oberon microcomputer, which was one of the first computers to be released with an operating system. The Oberon OS used Component Pascal as its programming language. Component Pascal is a modernized version of Oberon Microsystem's original Pascal implementation. It has been enhanced and extended over time by several developers at BlackBox Software GmbH. In particular, it has gained support for object-oriented programming (since 1995) and improved support for",Oberon Microsystems<|im_end|>
`Log.String(`,"Log.String(""Hello World!"") (* синтаксис: Log.String('') *) Вывод строки в журнал. Строка может содержать любые символы, которые могут быть приняты языком программирования. Для вывода литерных символов необходимо использовать одиночный кавычки (''), а для чисел и других специальных символов (например, пробелов) - двойные (''). Пример: ",Log.Int(n)<|im_end|>
`Типом целой константы является`,"тип INTEGER. Константа может быть названа как переменной, но не может быть изменена после инициализации. CONST min, max Минимальное и максимальное значения для диапазона значений целых чисел. CONST 1, 2, 4, 8, 16, 32, 64, 128, 256, 512, 1024,","CPM.Mark(126, pos)<|im_end|>"


In [12]:
draw_diff(False, 39)

USING CHAT TEMPLATE = False


PROMPT,BEFORE,AFTER
`Как в BlackBox можно вывести данные на консоль?`,"Как это делается? Для начала, если у вас есть какая-то программа, которая выводит данные на консоль (например, текст с помощью TextCmds), то",<|im_end|>
`Какой тип данных в Component Pascal используется для хранения целых чисел?`,"В каком диапазоне значений может быть представлено такое число? Чтобы ответить на эти вопросы, нужно сначала понять, что такое целые числа","- Int16, Int32, Char8, Char16, String16, Real32 и Real64. В каких случаях используются эти типы? Как"
`МОДУЛЬ i21егэDemo2010C4ru;`,(* info21 2010-09-18 *) ПОДКЛЮЧИТЬ вьанов2003 := Загрузить,(**) *END; (**) MODULE i21егэDemo2010C4ru; (**) (*<
`Component Pascal is Oberon microsystems refinement of?`,"It is a simple extension of the BlackBox Component Framework. The difference lies in the way that the modules are organized and how they are used. In particular, it allows to use modules which are","Yes, it is a refinement. The reason for this is that the original Oberon microsystem does not support the use of a pointer type in Component Pascal. This restriction has been removed in Component"
`Log.String(`,"10, 2, "" "", TRUE); Log.Int(3) END Do; END ObxCount1. Listing 5-1. Counting the digits of a number","""String"" ); StdLog.Int( n ); StdLog.Ln; END ``` Используется для вывода строки, которая содержит значение переменной `n`. После этого выз"
`Типом целой константы является`,"INTEGER. См. также модуль DevCPT. CONST byte, char, 16 Эти значения используются в качестве значений для параметра mode (см. выше","Ntype. Имя типа: Int8..Int32, Char16, String8 и Char16. В случае неопределенного типа используется тип DevCPT.NIL."


In [24]:
print(infer(model, 'Ты помошник по среде Component Pascal. Ответь на вопрос: Как мне вывести строку hello в окно лога?', 256, use_chat = True))

Для вывода строки в окно лога нужно использовать процедуру StdLog.String, которая принимает параметр типа String и помещает его содержимое в окно лога. Следующий пример демонстрирует эту процедуру:

MODULE  i21примЛог;

	IMPORT  Log := StdLog;
	
	PROCEDURE Показать* ( s: ARRAY OF CHAR );
	BEGIN
		Log.String( s ); (* или просто Log.out *)
	END Показать;
	
END  i21примЛог.

Выводится следующее:
	i21примЛог.Показать("hello")

См. также модуль Log для более подробной информации о работе с логом.



In [8]:
checkpoint = transformers.AutoModelForCausalLM.from_pretrained("./results/20250122-220441/checkpoint-1220/", device_map=device)

In [14]:
print(infer(checkpoint, 'Какой тип использовать для хранения целых чисел?', 256, use_chat = True))

Для целых чисел обычно используют тип INTEGER. Однако, если значения могут быть только некоторым предопределенным количеством значений, следует использовать тип SHORTINT, чтобы избежать возможных потерь точности, которые могут возникнуть при делении на 2. В некоторых случаях лучше использовать LONGINT.

Все последующие примеры использует LONGINT в качестве основного типа. Если результаты вычислений становятся непредсказуемыми, если используется другой тип.

Пример:
	
	PROCEDURE Ddouble* (a, b: LONGINT);
	BEGIN
		DIAGNOSTIC;
		StdLog.Nl;
		StdLog.String("Ddouble "); StdLog.Int(a + b, 0); StdLog.Ln;
		StdLog.String(DefaCT(a + b)); StdLog.Ln
	END Ddouble;

В этом примере использовано значение эффекта (defacto) стандартного модуля Math, который включает в себя итоговое значение.
После применения команд DIAGNOSTIC (см. ниже), он будет выровнен по левому краю, а не
